In [1]:
from pyspark.sql import SparkSession, DataFrame

In [ ]:
import geopandas as gpd
import pyproj
import polar as pl
import pandas as pd
import folium
from folium.plugins import HeatMap

shapefile_path = "app/map/taxi_zones.shp"

gdf = gpd.read_file(shapefile_path)
# Define
source_crs = gdf.crs  # CRS of the shapefile
target_crs = 'EPSG:4326'  # WGS84 - lat/lon CRS
transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)

gdf['longitude'] = gdf.geometry.centroid.x
gdf['latitude'] = gdf.geometry.centroid.y
gdf['longitude'], gdf['latitude'] = transformer.transform(gdf['longitude'], gdf['latitude'])

df = pd.DataFrame(gdf[['LocationID', 'longitude', 'latitude']])
print(df)

In [2]:
spark = (
            SparkSession.builder.master("local[*]")
            .appName("SparkByExamples.com")
            .getOrCreate()
)

23/12/19 15:31:21 WARN Utils: Your hostname, TrungUbun resolves to a loopback address: 127.0.1.1; using 192.168.1.6 instead (on interface enp3s0)
23/12/19 15:31:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/19 15:31:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/12/19 15:31:27 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [3]:
df = (
        spark.read.format("parquet")
        .options(header=True, inferSchema=False)
        .load("/home/trung/Downloads/gold_dropoff")
    )

In [4]:
df.show()

+-------------------+------------+--------------------+
|   Dropoff_datetime|DOLocationID|           DropOffID|
+-------------------+------------+--------------------+
|2023-01-22 07:49:01|        75.0|Y2023012225769803776|
|2023-01-22 07:56:41|       236.0|Y2023012225769803777|
|2023-01-22 07:54:23|       143.0|Y2023012225769803778|
|2023-01-22 07:56:39|        42.0|Y2023012225769803779|
|2023-01-22 07:31:54|        79.0|Y2023012225769803780|
|2023-01-22 07:51:12|       144.0|Y2023012225769803781|
|2023-01-22 07:06:24|       151.0|Y2023012225769803782|
|2023-01-22 07:56:49|        48.0|Y2023012225769803783|
|2023-01-22 07:42:28|       262.0|Y2023012225769803784|
|2023-01-22 07:52:45|       158.0|Y2023012225769803785|
|2023-01-22 07:12:28|       246.0|Y2023012225769803786|
|2023-01-22 07:52:17|       143.0|Y2023012225769803787|
|2023-01-22 08:04:53|        90.0|Y2023012225769803788|
|2023-01-22 07:27:46|        50.0|Y2023012225769803789|
|2023-01-22 07:25:06|       151.0|Y2023012225769

In [26]:
df = (
        spark.read.format("parquet")
        .options(header=True, inferSchema=False)
        .load("/home/trung/Downloads/a/green_tripdata_2023-02.parquet")
    )
# count row have tpep_pickup_datetime null
print(df.filter(df.lpep_pickup_datetime.isNull()).count())
# count row have tpep_dropoff_datetime null
print(df.filter(df.lpep_dropoff_datetime.isNull()).count())
# count row have PULocationID null
print(df.filter(df.PULocationID.isNull()).count())
# count row have DOLocationID null
print(df.filter(df.DOLocationID.isNull()).count())

0
0
0
0


In [22]:
df.count()

2913955

In [19]:
# count row have dispatching_base_num null
print(df.filter(df.dispatching_base_num.isNull()).count())
# count row have affiliated_base_number null
print(df.filter(df.Affiliated_base_number.isNull()).count())
# count row have sr_flag null
print(df.filter(df.SR_Flag.isNull()).count())

0
0
1110797


In [29]:
# count row have passenger_count null
print(df.filter(df.passenger_count.isNull()).count())
# count row have trip_distance null
print(df.filter(df.trip_distance.isNull()).count())
# count row have store_and_fwd_flag null
print(df.filter(df.store_and_fwd_flag.isNull()).count())
# count row have trip_type null
print(df.filter(df.trip_type.isNull()).count())
# count row have VendorID null
print(df.filter(df.VendorID.isNull()).count())


4821
0
4821
4826
0


passenger count, store_and_fwd_flag

In [28]:
# count row have fare_amount null
print(df.filter(df.fare_amount.isNull()).count())
# count row have mta_tax null
print(df.filter(df.mta_tax.isNull()).count())
# count row have improvement_surcharge null
print(df.filter(df.improvement_surcharge.isNull()).count())
# count row have payment_type null
print(df.filter(df.payment_type.isNull()).count())
# count row have RatecodeID null
print(df.filter(df.RatecodeID.isNull()).count())
# count row have tip_amount null
print(df.filter(df.tip_amount.isNull()).count())
# count row have tolls_amount null
print(df.filter(df.tolls_amount.isNull()).count())
# count row have total_amount null
print(df.filter(df.total_amount.isNull()).count())
# count row have congestion_surcharge null
print(df.filter(df.congestion_surcharge.isNull()).count())



0
0
0
4821
4821
0
0
0
4821


payment_type, RatecodeID, congestion_surcharge

In [9]:
df = (
        spark.read.format("parquet")
        .options(header=True, inferSchema=False)
        # .schema(trip_schema)
        .load("/home/trung/Downloads/gold_pickup")
    )
#print count null row in PULocationID
print(df.count())
df.filter(df.PULocationID.isNull()).count()

17273410


2914436